In [1]:
clean_up = True
%run stdPackages.ipynb

The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb7.gdx is still active and was not deleted.
The file _gams_py_gdb9.gdx is still active and was not deleted.


In [2]:
import math, dash
import plotly.express as px
from plotly.subplots import make_subplots

# Simulations in partial equilibrium

In [3]:
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
with open(f"{d['data']}\\AHH","rb") as file:
    H = pickle.load(file)
H.write();

Define a standardized report for a shock in this module as follows:
* Add a database with target values for an exogenous variable. Extract data on ```qD, pD, p``` in a solution loop.
* Plot time series for durables/non-durables on (sector, goods levels).
* Plot time series for durables/non-durables on goods levels.
* Plot time series for equilibrium and effective input prices. 

In [4]:
def stdPlot(df, c, title):
    nplots,nrows = len(c), math.ceil(len(c)/2)
    fig, axes = plt.subplots(nrows, min(nplots,2), figsize=(14,(4*nrows)));
    for i in range(nplots):
        ax = plt.subplot(nrows,min(nplots,2),i+1)
        df.xs(c[i], level = c.names).plot(ax=ax, legend=False);
        ax.set_title(c[i])
    fig.legend(df.columns,ncol=3,loc=8,frameon=True)
    # fig.suptitle(title,fontsize=15)
    fig.tight_layout()
    fig.subplots_adjust(bottom=0.30/nrows);
    return fig

In [5]:
def stdPlots(m,P):
    mult_graphs()
    sn = adj.rc_pd(P.get('input'), ('not', P.get('inv')))
    figs = {'qD_Durables': stdPlot(adj.rc_pd(m.out_db.get('sol_qD_shock'), P.get('dur')).unstack('l1'), P.get('dur'), "$q$, durables"),
            'pD_Durables': stdPlot(adj.rc_pd(m.out_db.get('sol_pD_shock'), ('and', [P.get('dur'), P.get('txE')])).unstack('l1'), P.get('dur'), "$p$, durable"),
            'qD_nonDurable': stdPlot(adj.rc_pd(m.out_db.get('sol_qD_shock'), ('and', [sn, P.get('txE')])).unstack('l1'), sn, "$q$, non-durables"),
            'p_equilibrium': stdPlot(adj.rc_pd(m.out_db.get('sol_p_shock'), ('and', [P.get('output_n'), P.get('txE')])).unstack('l1'), P.get('output_n'), "$p$, equilibrium"),
            'pD_nonDurable': stdPlot(adj.rc_pd(m.out_db.get('sol_pD_shock'), ('and', [P.get('output'), P.get('txE')])).unstack('l1'), P.get('output'), "$p$, non-durables")}
    return figs

#### 1. Increase an input tax

Increase tax on wooferman:

In [8]:
H.get('disc')

s
HH    0.970874
Name: level, dtype: object

*i. Set up target database:*

In [6]:
dbT = GpyDB(**{'name': 'TaxShock'})
dbT['tauD'] = pd.Series(1, index = pd.MultiIndex.from_product([P.get('t'), P.get('s'), pd.Index(['a'], name = 'n')]), name = 'tauD')
extractSol = {k:None for k in ('qD','pD','p')}

*ii. Solve:*

In [7]:
m_a, cp_a = P.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol, 'n': 3}, **{'cns': 'CONOPT4'})